# Analysis of unsupervised models

In [ ]:
import numpy as np
import pandas as pd
import respiration.utils as utils

evaluation_dir = utils.dir_path('outputs', 'signals')

predictions_file = utils.join_paths(evaluation_dir, 'unsupervised_predictions.csv')
predictions = pd.read_csv(predictions_file)
predictions['signal'] = predictions['signal'].apply(eval).apply(np.array)
predictions['duration'] = pd.to_timedelta(predictions['duration'])
predictions.head()

## Compare the predictions to the ground truth

In [ ]:
import os
import tqdm.auto as tqdm
import respiration.analysis as analysis

from respiration.dataset import VitalCamSet

analysis_dir = utils.dir_path('outputs', 'analysis', mkdir=True)
analysis_file = utils.join_paths(analysis_dir, 'unsupervised_analysis.csv')

dataset = VitalCamSet()

if os.path.exists(analysis_file):
    analysis_results = pd.read_csv(analysis_file)
else:
    analysis_results = []

    for index, row in tqdm.tqdm(predictions.iterrows(), total=len(predictions)):
        subject, setting = row['subject'], row['setting']

        # Get the ground truth signal and sampling rate
        gt_signal = dataset.get_breathing_signal(subject, setting)

        prediction, sampling_rate = row['signal'], row['sampling_rate']

        comparator = analysis.SignalComparator(
            prediction,
            gt_signal,
            sampling_rate,
        )

        for metric, result in comparator.all_results().items():
            analysis_results.append({
                'subject': subject,
                'setting': setting,
                'method': row['method'],
                'roi': row['roi'],
                'metric': metric,
                'prediction': result['prediction'],
                'ground_truth': result['ground_truth'],
            })

    analysis_results = pd.DataFrame(analysis_results)
    analysis_results.to_csv(analysis_file, index=False)

In [ ]:
analysis_results